In [ ]:
# ONS (Census 2011) statistics for Lewisham.
# Fetches Nomis data releases and prepares them for use in Lewisham ward profiles.
#
# This requires the OA->LSOA lookup table produced by "lbl_boundaries_oa.ipynb"
# as well as the OA->WD22 lookup table produced by "lbl_boundaries_2022_wards.ipynb"

In [ ]:
import pandas as pd

from google.colab import drive
import google.colab.files as files

import IPython

Tools
==
Helpers to reformat & aggregate the Nomis source data.

In [ ]:
# Extract the subset we need.
def filter_nomis_data(d, date_values, geography_values):
  return d[d.date.isin(date_values) & 
           d.geography.isin(geography_values)]

# Reformat Nomis data into a lookup table format:
# clear labelling of the index (geography) column, 
# remove superfluous columns.
def format_nomis_data(d, geo_colname='OA11CD'):
  return d.\
           rename(columns={'geography code': geo_colname}).\
           drop(columns=['date', 'geography'])

# Convenience function to call the above.
def nomis_to_oa(d, years, oa_list):
  return format_nomis_data(filter_nomis_data(d, years, oa_list))

In [ ]:
# Aggregate OA-level data to LSOAs, Wards, etc. (Calculates the sum.)
def aggregate_oa_groups(oa_data, oa_group_table, oa_col, group_col):
  return pd.merge(oa_group_table, oa_data, on=oa_col, how='right').\
              drop(columns=[oa_col]).\
              groupby(group_col).sum()

Data
==
GDrive mount
--

In [ ]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# Used as root folder.
project_dir = '/content/gdrive/MyDrive/WardProfiles'
oa_lookups_dir = f"{project_dir}/lookups/oa"
ward_lookups_dir = f"{project_dir}/lookups/2022_wards"

# For exports
output_dir = f"{project_dir}/housing"

In [ ]:
!mkdir -p '{output_dir}'

Downloads
--

In [ ]:
# Accommodation type (available from table: Dwellings, household spaces and accommodation type)
# Source: 
# https://www.nomisweb.co.uk/census/2011/ks401ew
!wget 'https://www.nomisweb.co.uk/api/v01/dataset/nm_618_1.bulk.csv?time=latest&measures=20100&rural_urban=total&geography=2013265927TYPE299' \
  -O "{output_dir}/census11_accommodation_type_oa11_london.csv"

--2021-11-20 11:21:54--  https://www.nomisweb.co.uk/api/v01/dataset/nm_618_1.bulk.csv?time=latest&measures=20100&rural_urban=total&geography=2013265927TYPE299
Resolving www.nomisweb.co.uk (www.nomisweb.co.uk)... 129.234.253.212
Connecting to www.nomisweb.co.uk (www.nomisweb.co.uk)|129.234.253.212|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1951298 (1.9M) [text/csv]
Saving to: ‘/content/gdrive/MyDrive/WardProfiles/housing/census11_accommodation_type_oa11_london.csv’

/content/gdrive/MyD 100%[===================>]   1.86M   644KB/s    in 3.0s    

2021-11-20 11:22:10 (644 KB/s) - ‘/content/gdrive/MyDrive/WardProfiles/housing/census11_accommodation_type_oa11_london.csv’ saved [1951298/1951298]



In [ ]:
# Tenure - Households
# Source:
# https://www.nomisweb.co.uk/census/2011/qs405ew
!wget 'https://www.nomisweb.co.uk/api/v01/dataset/nm_537_1.bulk.csv?time=latest&measures=20100&rural_urban=total&geography=2013265927TYPE299' \
  -O "{output_dir}/census11_tenure_households_oa11_london.csv"

--2021-11-20 11:22:16--  https://www.nomisweb.co.uk/api/v01/dataset/nm_537_1.bulk.csv?time=latest&measures=20100&rural_urban=total&geography=2013265927TYPE299
Resolving www.nomisweb.co.uk (www.nomisweb.co.uk)... 129.234.253.212
Connecting to www.nomisweb.co.uk (www.nomisweb.co.uk)|129.234.253.212|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1880966 (1.8M) [text/csv]
Saving to: ‘/content/gdrive/MyDrive/WardProfiles/housing/census11_tenure_households_oa11_london.csv’

/content/gdrive/MyD 100%[===================>]   1.79M   644KB/s    in 2.9s    

2021-11-20 11:22:31 (644 KB/s) - ‘/content/gdrive/MyDrive/WardProfiles/housing/census11_tenure_households_oa11_london.csv’ saved [1880966/1880966]



In [ ]:
# Household Composition
# Source: 
# https://www.nomisweb.co.uk/census/2011/ks105ew
!wget 'https://www.nomisweb.co.uk/api/v01/dataset/nm_605_1.bulk.csv?time=latest&measures=20100&rural_urban=total&geography=2013265927TYPE299' \
  -O "{output_dir}/census11_household_compositions_oa11_london.csv"

--2021-11-20 11:22:36--  https://www.nomisweb.co.uk/api/v01/dataset/nm_605_1.bulk.csv?time=latest&measures=20100&rural_urban=total&geography=2013265927TYPE299
Resolving www.nomisweb.co.uk (www.nomisweb.co.uk)... 129.234.253.212
Connecting to www.nomisweb.co.uk (www.nomisweb.co.uk)|129.234.253.212|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2379863 (2.3M) [text/csv]
Saving to: ‘/content/gdrive/MyDrive/WardProfiles/housing/census11_household_compositions_oa11_london.csv’

/content/gdrive/MyD 100%[===================>]   2.27M   641KB/s    in 3.6s    

2021-11-20 11:22:53 (641 KB/s) - ‘/content/gdrive/MyDrive/WardProfiles/housing/census11_household_compositions_oa11_london.csv’ saved [2379863/2379863]



In [ ]:
# Rooms, bedrooms, central heating
# Source:
# https://www.nomisweb.co.uk/census/2011/ks403ew
!wget 'https://www.nomisweb.co.uk/api/v01/dataset/nm_146_1.bulk.csv?time=latest&measures=20100&rural_urban=total&geography=2013265927TYPE299' \
  -O "{output_dir}/census11_rooms_bedrooms_central_heating_oa11_london.csv"

--2021-11-20 11:22:59--  https://www.nomisweb.co.uk/api/v01/dataset/nm_146_1.bulk.csv?time=latest&measures=20100&rural_urban=total&geography=2013265927TYPE299
Resolving www.nomisweb.co.uk (www.nomisweb.co.uk)... 129.234.253.212
Connecting to www.nomisweb.co.uk (www.nomisweb.co.uk)|129.234.253.212|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1662827 (1.6M) [text/csv]
Saving to: ‘/content/gdrive/MyDrive/WardProfiles/housing/census11_rooms_bedrooms_central_heating_oa11_london.csv’

/content/gdrive/MyD 100%[===================>]   1.58M   630KB/s    in 2.6s    

2021-11-20 11:23:13 (630 KB/s) - ‘/content/gdrive/MyDrive/WardProfiles/housing/census11_rooms_bedrooms_central_heating_oa11_london.csv’ saved [1662827/1662827]



In [ ]:
# Car or van availability
# Source:
# https://www.nomisweb.co.uk/census/2011/ks404ew
!wget 'https://www.nomisweb.co.uk/api/v01/dataset/nm_621_1.bulk.csv?time=latest&measures=20100&rural_urban=total&geography=2013265927TYPE299' \
  -O "{output_dir}/census11_car_van_availability_oa11_london.csv"

--2021-11-20 11:23:18--  https://www.nomisweb.co.uk/api/v01/dataset/nm_621_1.bulk.csv?time=latest&measures=20100&rural_urban=total&geography=2013265927TYPE299
Resolving www.nomisweb.co.uk (www.nomisweb.co.uk)... 129.234.253.212
Connecting to www.nomisweb.co.uk (www.nomisweb.co.uk)|129.234.253.212|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1485244 (1.4M) [text/csv]
Saving to: ‘/content/gdrive/MyDrive/WardProfiles/housing/census11_car_van_availability_oa11_london.csv’

/content/gdrive/MyD 100%[===================>]   1.42M   367KB/s    in 4.6s    

2021-11-20 11:23:34 (318 KB/s) - ‘/content/gdrive/MyDrive/WardProfiles/housing/census11_car_van_availability_oa11_london.csv’ saved [1485244/1485244]



In [ ]:
# Occupancy rating (bedrooms)
# Source:
# https://www.nomisweb.co.uk/census/2011/qs412ew
!wget 'https://www.nomisweb.co.uk/api/v01/dataset/nm_544_1.bulk.csv?time=latest&measures=20100&rural_urban=total&geography=2013265927TYPE299' \
  -O "{output_dir}/census11_occupancy_rating_oa11_london.csv"

--2021-11-20 11:23:39--  https://www.nomisweb.co.uk/api/v01/dataset/nm_544_1.bulk.csv?time=latest&measures=20100&rural_urban=total&geography=2013265927TYPE299
Resolving www.nomisweb.co.uk (www.nomisweb.co.uk)... 129.234.253.212
Connecting to www.nomisweb.co.uk (www.nomisweb.co.uk)|129.234.253.212|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1412216 (1.3M) [text/csv]
Saving to: ‘/content/gdrive/MyDrive/WardProfiles/housing/census11_occupancy_rating_oa11_london.csv’

/content/gdrive/MyD 100%[===================>]   1.35M   632KB/s    in 2.2s    

2021-11-20 11:23:53 (632 KB/s) - ‘/content/gdrive/MyDrive/WardProfiles/housing/census11_occupancy_rating_oa11_london.csv’ saved [1412216/1412216]



In [ ]:
# reference regions

In [ ]:
# reference regions
!wget 'https://www.nomisweb.co.uk/api/v01/dataset/nm_618_1.bulk.csv?time=latest&measures=20100&rural_urban=total&geography=2092957699,2013265927TYPE480,1946157254TYPE464' \
  -O "{output_dir}/census11_accommodation_type_references.csv"

--2021-11-20 11:24:09--  https://www.nomisweb.co.uk/api/v01/dataset/nm_618_1.bulk.csv?time=latest&measures=20100&rural_urban=total&geography=2092957699,2013265927TYPE480,1946157254TYPE464
Resolving www.nomisweb.co.uk (www.nomisweb.co.uk)... 129.234.253.212
Connecting to www.nomisweb.co.uk (www.nomisweb.co.uk)|129.234.253.212|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1570 (1.5K) [text/csv]
Saving to: ‘/content/gdrive/MyDrive/WardProfiles/housing/census11_accommodation_type_references.csv’

/content/gdrive/MyD 100%[===================>]   1.53K  --.-KB/s    in 0s      

2021-11-20 11:24:10 (202 MB/s) - ‘/content/gdrive/MyDrive/WardProfiles/housing/census11_accommodation_type_references.csv’ saved [1570/1570]



In [ ]:
# tenure_households
!wget 'https://www.nomisweb.co.uk/api/v01/dataset/nm_537_1.bulk.csv?time=latest&measures=20100&rural_urban=total&geography=2092957699,2013265927TYPE480,1946157254TYPE464' \
  -O "{output_dir}/census11_tenure_households_references.csv"

--2021-11-20 11:24:34--  https://www.nomisweb.co.uk/api/v01/dataset/nm_537_1.bulk.csv?time=latest&measures=20100&rural_urban=total&geography=2092957699,2013265927TYPE480,1946157254TYPE464
Resolving www.nomisweb.co.uk (www.nomisweb.co.uk)... 129.234.253.212
Connecting to www.nomisweb.co.uk (www.nomisweb.co.uk)|129.234.253.212|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1281 (1.3K) [text/csv]
Saving to: ‘/content/gdrive/MyDrive/WardProfiles/housing/census11_tenure_households_references.csv’

/content/gdrive/MyD 100%[===================>]   1.25K  --.-KB/s    in 0s      

2021-11-20 11:24:35 (146 MB/s) - ‘/content/gdrive/MyDrive/WardProfiles/housing/census11_tenure_households_references.csv’ saved [1281/1281]



In [ ]:
# household_compositions
!wget 'https://www.nomisweb.co.uk/api/v01/dataset/nm_605_1.bulk.csv?time=latest&measures=20100&rural_urban=total&geography=2092957699,2013265927TYPE480,1946157254TYPE464' \
  -O "{output_dir}/census11_household_compositions_references.csv"

--2021-11-20 11:24:43--  https://www.nomisweb.co.uk/api/v01/dataset/nm_605_1.bulk.csv?time=latest&measures=20100&rural_urban=total&geography=2092957699,2013265927TYPE480,1946157254TYPE464
Resolving www.nomisweb.co.uk (www.nomisweb.co.uk)... 129.234.253.212
Connecting to www.nomisweb.co.uk (www.nomisweb.co.uk)|129.234.253.212|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2544 (2.5K) [text/csv]
Saving to: ‘/content/gdrive/MyDrive/WardProfiles/housing/census11_household_compositions_references.csv’

/content/gdrive/MyD 100%[===================>]   2.48K  --.-KB/s    in 0s      

2021-11-20 11:24:44 (187 MB/s) - ‘/content/gdrive/MyDrive/WardProfiles/housing/census11_household_compositions_references.csv’ saved [2544/2544]



In [ ]:
# rooms_bedrooms_central_heating
!wget 'https://www.nomisweb.co.uk/api/v01/dataset/nm_146_1.bulk.csv?time=latest&measures=20100&rural_urban=total&geography=2092957699,2013265927TYPE480,1946157254TYPE464' \
  -O "{output_dir}/census11_rooms_bedrooms_central_heating_references.csv"

--2021-11-20 11:24:50--  https://www.nomisweb.co.uk/api/v01/dataset/nm_146_1.bulk.csv?time=latest&measures=20100&rural_urban=total&geography=2092957699,2013265927TYPE480,1946157254TYPE464
Resolving www.nomisweb.co.uk (www.nomisweb.co.uk)... 129.234.253.212
Connecting to www.nomisweb.co.uk (www.nomisweb.co.uk)|129.234.253.212|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 885 [text/csv]
Saving to: ‘/content/gdrive/MyDrive/WardProfiles/housing/census11_rooms_bedrooms_central_heating_references.csv’

/content/gdrive/MyD 100%[===================>]     885  --.-KB/s    in 0s      

2021-11-20 11:24:52 (118 MB/s) - ‘/content/gdrive/MyDrive/WardProfiles/housing/census11_rooms_bedrooms_central_heating_references.csv’ saved [885/885]



In [ ]:
# car_van_availability
!wget 'https://www.nomisweb.co.uk/api/v01/dataset/nm_621_1.bulk.csv?time=latest&measures=20100&rural_urban=total&geography=2092957699,2013265927TYPE480,1946157254TYPE464' \
  -O "{output_dir}/census11_car_van_availability_references.csv"

--2021-11-20 11:24:57--  https://www.nomisweb.co.uk/api/v01/dataset/nm_621_1.bulk.csv?time=latest&measures=20100&rural_urban=total&geography=2092957699,2013265927TYPE480,1946157254TYPE464
Resolving www.nomisweb.co.uk (www.nomisweb.co.uk)... 129.234.253.212
Connecting to www.nomisweb.co.uk (www.nomisweb.co.uk)|129.234.253.212|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 707 [text/csv]
Saving to: ‘/content/gdrive/MyDrive/WardProfiles/housing/census11_car_van_availability_references.csv’

/content/gdrive/MyD 100%[===================>]     707  --.-KB/s    in 0s      

2021-11-20 11:24:59 (90.6 MB/s) - ‘/content/gdrive/MyDrive/WardProfiles/housing/census11_car_van_availability_references.csv’ saved [707/707]



In [ ]:
# occupancy_rating
!wget 'https://www.nomisweb.co.uk/api/v01/dataset/nm_544_1.bulk.csv?time=latest&measures=20100&rural_urban=total&geography=2092957699,2013265927TYPE480,1946157254TYPE464' \
  -O "{output_dir}/census11_occupancy_rating_references.csv"

--2021-11-20 11:25:03--  https://www.nomisweb.co.uk/api/v01/dataset/nm_544_1.bulk.csv?time=latest&measures=20100&rural_urban=total&geography=2092957699,2013265927TYPE480,1946157254TYPE464
Resolving www.nomisweb.co.uk (www.nomisweb.co.uk)... 129.234.253.212
Connecting to www.nomisweb.co.uk (www.nomisweb.co.uk)|129.234.253.212|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 736 [text/csv]
Saving to: ‘/content/gdrive/MyDrive/WardProfiles/housing/census11_occupancy_rating_references.csv’

/content/gdrive/MyD 100%[===================>]     736  --.-KB/s    in 0s      

2021-11-20 11:25:05 (59.1 MB/s) - ‘/content/gdrive/MyDrive/WardProfiles/housing/census11_occupancy_rating_references.csv’ saved [736/736]



Lookups
--
Used to match up OAs with their LSOAs, and 2022 electoral wards

In [ ]:
# All OAs and their LSOAs in Lewisham
oa_lsoa_join = pd.read_csv(f"{oa_lookups_dir}/lbl_oa11_lsoa11_msoa11_lad20_rgn20_202012.csv")
oa_lsoa_join = oa_lsoa_join[['OA11CD', 'LSOA11CD']]
oa_lsoa_join.head()

,OA11CD,LSOA11CD
0,E00016277,E01003220
1,E00016278,E01003220
2,E00016285,E01003220
3,E00016257,E01003221
4,E00016263,E01003221


In [ ]:
# All OAs and their 2022 wards in Lewisham
oa_wd22_join = pd.read_csv(f"{ward_lookups_dir}/lbl_oa11_wd22_proposed.csv")
oa_wd22_join = oa_wd22_join[['OA11CD', 'WD22CD_proposed']]
oa_wd22_join.head()

,OA11CD,WD22CD_proposed
0,E00016403,E05013721
1,E00016442,E05013721
2,E00016407,E05013721
3,E00016404,E05013721
4,E00016402,E05013721


Process & export
==
Lewisham
--

In [ ]:
# For filtering of source data
lbl_oa_list = oa_lsoa_join.OA11CD.unique() # All OAs in Lewisham
years = [2011] # Just a precaution, in case any of the downloads include multiple periods

for datname in ['accommodation_type', 'tenure_households',
                'household_compositions', 'rooms_bedrooms_central_heating',
                'car_van_availability', 'occupancy_rating']:
  IPython.display.display(f"=== {datname} ===")
  # Load the Nomis data
  d = pd.read_csv(f"{output_dir}/census11_{datname}_oa11_london.csv")

  # Simplify the column names
  d.columns = d.columns.str.replace(
      # Remove the redundant title prefix and qualifier suffix
      r'^.*?: (.*?); measures: Value', 
      r'\1')

  # Transform to OA index data
  lbl_oa = nomis_to_oa(d, years, lbl_oa_list)
  lbl_oa.to_csv(f"{output_dir}/lbl_{datname}_oa11.csv", index=False)

  # Aggregate to LSOA level
  lbl_lsoa = aggregate_oa_groups(lbl_oa, oa_lsoa_join, oa_col='OA11CD', group_col='LSOA11CD')
  lbl_lsoa.to_csv(f"{output_dir}/lbl_{datname}_lsoa11.csv", index=True)
  IPython.display.display(lbl_lsoa.head())

  # Aggregate to WD22 level
  lbl_wd22 = aggregate_oa_groups(lbl_oa, oa_wd22_join, oa_col='OA11CD', group_col='WD22CD_proposed')
  lbl_wd22.to_csv(f"{output_dir}/lbl_{datname}_wd22.csv", index=True)
  IPython.display.display(lbl_wd22.head())

'=== accommodation_type ==='

,All categories: Dwelling type,Unshared dwelling,Shared dwelling: Two household spaces,Shared dwelling: Three or more household spaces,All categories: Household spaces,Household spaces with at least one usual resident,Household spaces with no usual residents,Whole house or bungalow: Detached,Whole house or bungalow: Semi-detached,Whole house or bungalow: Terraced (including end-terrace),"Flat, maisonette or apartment: Purpose-built block of flats or tenement","Flat, maisonette or apartment: Part of a converted or shared house (including bed-sits)","Flat, maisonette or apartment: In a commercial building",Caravan or other mobile or temporary structure
LSOA11CD,,,,,,,,,,,,,,
E01003189,628,628,0,0,628,624,4,21,155,261,164,26,1,0
E01003190,819,818,0,1,822,809,13,9,70,120,578,34,11,0
E01003191,676,676,0,0,676,665,11,19,90,145,380,41,1,0
E01003192,749,749,0,0,749,741,8,16,83,154,468,26,2,0
E01003193,672,670,1,1,678,666,12,12,125,331,187,14,9,0


,All categories: Dwelling type,Unshared dwelling,Shared dwelling: Two household spaces,Shared dwelling: Three or more household spaces,All categories: Household spaces,Household spaces with at least one usual resident,Household spaces with no usual residents,Whole house or bungalow: Detached,Whole house or bungalow: Semi-detached,Whole house or bungalow: Terraced (including end-terrace),"Flat, maisonette or apartment: Purpose-built block of flats or tenement","Flat, maisonette or apartment: Part of a converted or shared house (including bed-sits)","Flat, maisonette or apartment: In a commercial building",Caravan or other mobile or temporary structure
WD22CD_proposed,,,,,,,,,,,,,,
E05013714,4395,4394,0,1,4398,4354,44,123,898,1271,1859,215,32,0
E05013715,7184,7162,7,15,7234,7070,164,310,627,1245,3458,1459,134,1
E05013716,7268,7224,22,22,7359,7203,156,154,479,1546,2573,2512,94,1
E05013717,6196,6178,4,14,6247,6145,102,189,2135,1981,1056,829,54,3
E05013718,6389,6379,6,4,6411,6263,148,204,852,2593,1216,1356,189,1


'=== tenure_households ==='

,All categories: Tenure,Owned: Total,Owned: Owned outright,Owned: Owned with a mortgage or loan,Shared ownership (part owned and part rented),Social rented: Total,Social rented: Rented from council (Local Authority),Social rented: Other social rented,Private rented: Total,Private rented: Private landlord or letting agency,Private rented: Employer of a household member,Private rented: Relative or friend of household member,Private rented: Other,Living rent free
LSOA11CD,,,,,,,,,,,,,,
E01003189,624,212,55,157,2,343,115,228,60,52,0,6,2,7
E01003190,809,233,79,154,2,444,337,107,123,116,0,5,2,7
E01003191,665,268,98,170,20,277,55,222,96,85,0,7,4,4
E01003192,741,196,58,138,12,449,167,282,77,68,1,6,2,7
E01003193,666,368,147,221,17,203,163,40,72,69,0,3,0,6


...

,All categories: Car or van availability,No cars or vans in household,1 car or van in household,2 cars or vans in household,3 cars or vans in household,4 or more cars or vans in household,sum of all cars or vans in the area
LSOA11CD,,,,,,,
E01003189,624,300,252,56,14,2,414
E01003190,809,421,324,56,3,5,473
E01003191,665,306,282,64,9,4,453
E01003192,741,412,280,42,7,0,385
E01003193,666,266,302,78,15,5,523


,All categories: Car or van availability,No cars or vans in household,1 car or van in household,2 cars or vans in household,3 cars or vans in household,4 or more cars or vans in household,sum of all cars or vans in the area
WD22CD_proposed,,,,,,,
E05013714,4354,2126,1769,376,67,16,2799
E05013715,7070,3251,3060,656,74,29,4733
E05013716,7203,4110,2590,429,59,15,3691
E05013717,6145,2097,2772,1004,216,56,5669
E05013718,6263,2610,2846,664,115,28,4652


'=== occupancy_rating ==='

,All categories: Occupancy rating bedrooms,Occupancy rating (bedrooms) of +2 or more,Occupancy rating (bedrooms) of +1,Occupancy rating (bedrooms) of 0,Occupancy rating (bedrooms) of -1,Occupancy rating (bedrooms) of -2 or less
LSOA11CD,,,,,,
E01003189,624,101,157,275,79,12
E01003190,809,65,241,368,118,17
E01003191,665,65,253,270,68,9
E01003192,741,80,181,377,83,20
E01003193,666,175,171,251,61,8


,All categories: Occupancy rating bedrooms,Occupancy rating (bedrooms) of +2 or more,Occupancy rating (bedrooms) of +1,Occupancy rating (bedrooms) of 0,Occupancy rating (bedrooms) of -1,Occupancy rating (bedrooms) of -2 or less
WD22CD_proposed,,,,,,
E05013714,4354,566,1256,1922,508,102
E05013715,7070,1422,2147,2871,538,92
E05013716,7203,904,1815,3494,826,164
E05013717,6145,1890,1716,1988,449,102
E05013718,6263,1531,2040,2115,502,75


Reference geographies
--

In [ ]:
# For filtering of source data
reference_names = ['United Kingdom', 'Great Britain', 'England and Wales',
                   'England', 'London', 'Lewisham']
years = [2011] # Just a precaution, in case any of the downloads include multiple periods

for datname in ['accommodation_type', 'tenure_households', 
                'household_compositions', 'rooms_bedrooms_central_heating',
                'car_van_availability', 'occupancy_rating']:
  IPython.display.display(f"=== {datname} ===")
  # Load the Nomis data
  d = pd.read_csv(f"{output_dir}/census11_{datname}_references.csv")

  # Simplify the column names
  d.columns = d.columns.str.replace(
      # Remove the redundant title prefix and qualifier suffix
      r'^.*?: (.*?); measures: Value', 
      r'\1')

  # Filter & transform to index data
  ref = filter_nomis_data(d, years, reference_names).drop(columns=['date'])
  ref = ref.rename(columns={'geography': 'Name', 'geography code': 'Code'})
  ref.to_csv(f"{output_dir}/references_{datname}.csv", index=False)
  IPython.display.display(ref.head())

'=== accommodation_type ==='

,Name,Code,Rural Urban,All categories: Dwelling type,Unshared dwelling,Shared dwelling: Two household spaces,Shared dwelling: Three or more household spaces,All categories: Household spaces,Household spaces with at least one usual resident,Household spaces with no usual residents,Whole house or bungalow: Detached,Whole house or bungalow: Semi-detached,Whole house or bungalow: Terraced (including end-terrace),"Flat, maisonette or apartment: Purpose-built block of flats or tenement","Flat, maisonette or apartment: Part of a converted or shared house (including bed-sits)","Flat, maisonette or apartment: In a commercial building",Caravan or other mobile or temporary structure
0,England,E92000001,Total,22976066,22955448,6700,13918,23044097,22063368,980729,5128552,7076395,5642969,3854451,984284,257218,100228
1,London,E12000007,Total,3358163,3349011,3160,5992,3387255,3266173,121082,211232,629607,776821,1274526,429456,62795,2818
2,Lewisham,E09000023,Total,117651,117309,131,211,118617,116091,2526,3868,15588,33402,42669,21106,1932,52


'=== tenure_households ==='

,Name,Code,Rural Urban,All categories: Tenure,Owned: Total,Owned: Owned outright,Owned: Owned with a mortgage or loan,Shared ownership (part owned and part rented),Social rented: Total,Social rented: Rented from council (Local Authority),Social rented: Other social rented,Private rented: Total,Private rented: Private landlord or letting agency,Private rented: Employer of a household member,Private rented: Relative or friend of household member,Private rented: Other,Living rent free
0,England,E92000001,Total,22063368,13975024,6745584,7229440,173760,3903550,2079778,1823772,3715924,3401675,55211,199428,59610,295110
1,London,E12000007,Total,3266173,1576207,689898,886309,42108,785993,439727,346266,819085,775591,5410,27937,10147,42780
2,Lewisham,E09000023,Total,116091,49228,17273,31955,1436,36052,18084,17968,28216,26665,89,1139,323,1159


'=== household_compositions ==='

...

,Name,Code,Rural Urban,All categories: Type of central heating in household,Does not have central heating,Does have central heating,Occupancy rating (rooms) of -1 or less,Occupancy rating (bedrooms) of -1 or less,Average household size,Average number of rooms per household,Average number of bedrooms per household
0,England,E92000001,Total,22063368,594561,21468807,1928596,1024473,2.4,5.4,2.7
1,London,E12000007,Total,3266173,92411,3173762,707437,370531,2.5,4.7,2.5
2,Lewisham,E09000023,Total,116091,3780,112311,25722,14018,2.4,4.5,2.3


'=== car_van_availability ==='

,Name,Code,Rural Urban,All categories: Car or van availability,No cars or vans in household,1 car or van in household,2 cars or vans in household,3 cars or vans in household,4 or more cars or vans in household,sum of all cars or vans in the area
0,England,E92000001,Total,22063368,5691251,9301776,5441593,1203865,424883,25696833
1,London,E12000007,Total,3266173,1357251,1324032,458659,95619,30612,2664414
2,Lewisham,E09000023,Total,116091,55893,46991,10829,1875,503,76507


'=== occupancy_rating ==='

,Name,Code,Rural Urban,All categories: Occupancy rating bedrooms,Occupancy rating (bedrooms) of +2 or more,Occupancy rating (bedrooms) of +1,Occupancy rating (bedrooms) of 0,Occupancy rating (bedrooms) of -1,Occupancy rating (bedrooms) of -2 or less
0,England,E92000001,Total,22063368,7558815,7594129,5885951,870540,153933
1,London,E12000007,Total,3266173,689538,923221,1282883,301325,69206
2,Lewisham,E09000023,Total,116091,20981,32468,48624,11704,2314


Derivatives
==

Accommodation type
--

In [ ]:
accomm = pd.read_csv(f"{output_dir}/census11_accommodation_type_oa11_london.csv")
accomm.columns


Index(['date', 'geography', 'geography code', 'Rural Urban',
       'Dwelling Type: All categories: Dwelling type; measures: Value',
       'Dwelling Type: Unshared dwelling; measures: Value',
       'Dwelling Type: Shared dwelling: Two household spaces; measures: Value',
       'Dwelling Type: Shared dwelling: Three or more household spaces; measures: Value',
       'Dwelling Type: All categories: Household spaces; measures: Value',
       'Dwelling Type: Household spaces with at least one usual resident; measures: Value',
       'Dwelling Type: Household spaces with no usual residents; measures: Value',
       'Dwelling Type: Whole house or bungalow: Detached; measures: Value',
       'Dwelling Type: Whole house or bungalow: Semi-detached; measures: Value',
       'Dwelling Type: Whole house or bungalow: Terraced (including end-terrace); measures: Value',
       'Dwelling Type: Flat, maisonette or apartment: Purpose-built block of flats or tenement; measures: Value',
       'Dwelling

In [ ]:
# Segmentation of categories
def accomm_type_groups(accomm, fixed_columns):
  d = accomm[fixed_columns].copy()
  total = accomm['All categories: Household spaces']
  d['Detached houses'] = accomm['Whole house or bungalow: Detached'] * 100. / total
  d['Semi-detached houses'] = accomm['Whole house or bungalow: Semi-detached'] * 100. / total
  d['Terraced'] = accomm['Whole house or bungalow: Terraced (including end-terrace)'] * 100 / total
  d['Flats'] = accomm[accomm.columns[accomm.columns.str.contains('Flat, maisonette or apartment')]].sum(axis=1) * 100 / total
  d['Caravan or mobile'] = accomm['Caravan or other mobile or temporary structure'] * 100 / total

  
  return d#.round(1)

In [ ]:
# Lewisham
for geog, geog_colname in zip(['oa11', 'wd22'], ['OA11CD', 'WD22CD_proposed']):
  IPython.display.display(f"=== {geog} ===")

  accomm = pd.read_csv(f"{output_dir}/lbl_accommodation_type_{geog}.csv")
  d = accomm_type_groups(accomm, [geog_colname])
  d.to_csv(f"{output_dir}/lbl_accommodation_type_groups_{geog}.csv", index=False)
  IPython.display.display(d.head())

'=== oa11 ==='

,OA11CD,Detached houses,Semi-detached houses,Terraced,Flats,Caravan or mobile
0,E00016403,0.000000,0.684932,0.684932,98.630137,0.0
1,E00016437,0.000000,1.242236,2.484472,96.273292,0.0
2,E00016439,0.552486,0.552486,3.867403,95.027624,0.0
3,E00016442,0.689655,0.689655,2.068966,96.551724,0.0
4,E00016399,9.722222,20.833333,31.250000,38.194444,0.0


'=== wd22 ==='

,WD22CD_proposed,Detached houses,Semi-detached houses,Terraced,Flats,Caravan or mobile
0,E05013714,2.796726,20.418372,28.899500,47.885402,0.000000
1,E05013715,4.285319,8.667404,17.210395,69.823058,0.013824
2,E05013716,2.092676,6.509037,21.008289,70.376410,0.013589
3,E05013717,3.025452,34.176405,31.711221,31.038899,0.048023
4,E05013718,3.182031,13.289658,40.446108,43.066604,0.015598


Tenure Households
--

In [ ]:
ths = pd.read_csv(f"{output_dir}/census11_tenure_households_oa11_london.csv")
ths.columns

Index(['date', 'geography', 'geography code', 'Rural Urban',
       'Tenure: All categories: Tenure; measures: Value',
       'Tenure: Owned: Total; measures: Value',
       'Tenure: Owned: Owned outright; measures: Value',
       'Tenure: Owned: Owned with a mortgage or loan; measures: Value',
       'Tenure: Shared ownership (part owned and part rented); measures: Value',
       'Tenure: Social rented: Total; measures: Value',
       'Tenure: Social rented: Rented from council (Local Authority); measures: Value',
       'Tenure: Social rented: Other social rented; measures: Value',
       'Tenure: Private rented: Total; measures: Value',
       'Tenure: Private rented: Private landlord or letting agency; measures: Value',
       'Tenure: Private rented: Employer of a household member; measures: Value',
       'Tenure: Private rented: Relative or friend of household member; measures: Value',
       'Tenure: Private rented: Other; measures: Value',
       'Tenure: Living rent free; mea

In [ ]:
from IPython.display import display
ths.head()

,date,geography,geography code,Rural Urban,Tenure: All categories: Tenure; measures: Value,Tenure: Owned: Total; measures: Value,Tenure: Owned: Owned outright; measures: Value,Tenure: Owned: Owned with a mortgage or loan; measures: Value,Tenure: Shared ownership (part owned and part rented); measures: Value,Tenure: Social rented: Total; measures: Value,Tenure: Social rented: Rented from council (Local Authority); measures: Value,Tenure: Social rented: Other social rented; measures: Value,Tenure: Private rented: Total; measures: Value,Tenure: Private rented: Private landlord or letting agency; measures: Value,Tenure: Private rented: Employer of a household member; measures: Value,Tenure: Private rented: Relative or friend of household member; measures: Value,Tenure: Private rented: Other; measures: Value,Tenure: Living rent free; measures: Value
0,2011,E00004482,E00004482,Total,121,68,35,33,1,20,15,5,32,32,0,0,0,0
1,2011,E00004497,E00004497,Total,117,26,3,23,0,78,76,2,11,9,0,2,0,2
2,2011,E00004498,E00004498,Total,145,39,15,24,1,90,89,1,14,14,0,0,0,1
3,2011,E00004502,E00004502,Total,120,23,11,12,0,84,83,1,13,13,0,0,0,0
4,2011,E00004505,E00004505,Total,124,36,15,21,2,69,51,18,16,16,0,0,0,1


In [ ]:
# Segmentation of categories
def tenure_groups(ths, fixed_columns):
  d = ths[fixed_columns].copy()
  total = ths['All categories: Tenure']
  d['Owned'] = ths['Owned: Total'] * 100. / total
  d['Shared ownership'] = ths['Shared ownership (part owned and part rented)'] * 100. / total
  d['Social rented'] = ths['Social rented: Total'] * 100. / total
  d['Private rented'] = ths['Private rented: Total'] * 100 / total
  d['Rent free'] = ths['Living rent free'] * 100 / total

  
  return d#.round(1)

In [ ]:
# Lewisham
for geog, geog_colname in zip(['oa11', 'wd22'], ['OA11CD', 'WD22CD_proposed']):
  IPython.display.display(f"=== {geog} ===")

  ths = pd.read_csv(f"{output_dir}/lbl_tenure_households_{geog}.csv")
  d = tenure_groups(ths, [geog_colname])
  d.to_csv(f"{output_dir}/lbl_tenure_households_groups_{geog}.csv", index=False)
  IPython.display.display(d.head())

'=== oa11 ==='

,OA11CD,Owned,Shared ownership,Social rented,Private rented,Rent free
0,E00016403,5.633803,0.000000,87.323944,7.042254,0.000000
1,E00016437,12.578616,0.000000,79.874214,6.918239,0.628931
2,E00016439,4.494382,0.000000,71.348315,23.595506,0.561798
3,E00016442,10.071942,0.719424,79.136691,10.071942,0.000000
4,E00016399,6.382979,0.000000,87.234043,2.836879,3.546099


'=== wd22 ==='

,WD22CD_proposed,Owned,Shared ownership,Social rented,Private rented,Rent free
0,E05013714,33.670188,0.987598,52.319706,12.195682,0.826826
1,E05013715,41.499293,0.466761,30.381895,26.732673,0.919378
2,E05013716,33.569346,1.624323,28.099403,35.651812,1.055116
3,E05013717,63.515053,0.764849,17.396257,17.282343,1.041497
4,E05013718,55.021555,1.468945,16.844962,25.786364,0.878173


Household composition
--

In [ ]:
house_comp = pd.read_csv(f"{output_dir}/census11_household_compositions_oa11_london.csv")
house_comp.head()

,date,geography,geography code,Rural Urban,Household Composition: All categories: Household composition; measures: Value,Household Composition: One person household; measures: Value,Household Composition: One person household: Aged 65 and over; measures: Value,Household Composition: One person household: Other; measures: Value,Household Composition: One family household; measures: Value,Household Composition: One family only: All aged 65 and over; measures: Value,Household Composition: One family only: Married or same-sex civil partnership couple; measures: Value,Household Composition: One family only: Married or same-sex civil partnership couple: No children; measures: Value,Household Composition: One family only: Married or same-sex civil partnership couple: Dependent children; measures: Value,Household Composition: One family only: Married or same-sex civil partnership couple: All children non-dependent; measures: Value,Household Composition: One family only: Cohabiting couple; measures: Value,Household Composition: One family only: Cohabiting couple: No children; measures: Value,Household Composition: One family only: Cohabiting couple: Dependent children; measures: Value,Household Composition: One family only: Cohabiting couple: All children non-dependent; measures: Value,Household Composition: One family only: Lone parent; measures: Value,Household Composition: One family only: Lone parent: Dependent children; measures: Value,Household Composition: One family only: Lone parent: All children non-dependent; measures: Value,Household Composition: Other household types; measures: Value,Household Composition: Other household types: With dependent children; measures: Value,Household Composition: Other household types: All full-time students; measures: Value,Household Composition: Other household types: All aged 65 and over; measures: Value,Household Composition: Other household types: Other; measures: Value
0,2011,E00004482,E00004482,Total,121,33,9,24,70,6,36,9,17,10,16,11,4,1,12,9,3,18,3,1,1,13
1,2011,E00004497,E00004497,Total,117,43,8,35,63,3,18,6,9,3,18,12,4,2,24,16,8,11,5,0,0,6
2,2011,E00004498,E00004498,Total,145,45,14,31,85,6,25,9,15,1,10,7,3,0,44,31,13,15,4,0,0,11
3,2011,E00004502,E00004502,Total,120,49,14,35,52,3,14,5,7,2,9,6,1,2,26,17,9,19,6,1,3,9
4,2011,E00004505,E00004505,Total,124,37,12,25,65,3,27,8,16,3,9,7,1,1,26,19,7,22,4,0,0,18


In [ ]:
house_comp.columns

Index(['date', 'geography', 'geography code', 'Rural Urban',
       'Household Composition: All categories: Household composition; measures: Value',
       'Household Composition: One person household; measures: Value',
       'Household Composition: One person household: Aged 65 and over; measures: Value',
       'Household Composition: One person household: Other; measures: Value',
       'Household Composition: One family household; measures: Value',
       'Household Composition: One family only: All aged 65 and over; measures: Value',
       'Household Composition: One family only: Married or same-sex civil partnership couple; measures: Value',
       'Household Composition: One family only: Married or same-sex civil partnership couple: No children; measures: Value',
       'Household Composition: One family only: Married or same-sex civil partnership couple: Dependent children; measures: Value',
       'Household Composition: One family only: Married or same-sex civil partnership

In [ ]:
# Segmentation of categories (first segments: 3 groups)
def household_comp_groups(hh_comp, fixed_columns):
  d = hh_comp[fixed_columns].copy()
  total = hh_comp['All categories: Household composition']
  d['One person household'] = hh_comp['One person household'] * 100. / total
  d['One family household'] = hh_comp['One family household'] * 100. / total
  d['Other household types'] = hh_comp['Other household types'] * 100 / total
    
  return d#.round(1)

In [ ]:
# Lewisham
# first segments: 3 groups
for geog, geog_colname in zip(['oa11', 'wd22'], ['OA11CD', 'WD22CD_proposed']):
  IPython.display.display(f"=== {geog} ===")

  hh_comp = pd.read_csv(f"{output_dir}/lbl_household_compositions_{geog}.csv")
  d = household_comp_groups(hh_comp, [geog_colname])
  d.to_csv(f"{output_dir}/lbl_household_compositions_groups_{geog}.csv", index=False)
  IPython.display.display(d.head())

'=== oa11 ==='

,OA11CD,One person household,One family household,Other household types
0,E00016403,30.281690,53.521127,16.197183
1,E00016437,15.723270,64.779874,19.496855
2,E00016439,53.932584,29.213483,16.853933
3,E00016442,31.654676,53.956835,14.388489
4,E00016399,14.893617,70.921986,14.184397


'=== wd22 ==='

,WD22CD_proposed,One person household,One family household,Other household types
0,E05013714,34.175471,54.754249,11.070280
1,E05013715,37.553041,50.183876,12.263083
2,E05013716,36.776343,43.023740,20.199917
3,E05013717,30.024410,56.143206,13.832384
4,E05013718,30.544468,54.207249,15.248284


Rooms bedrooms central heating
--

In [ ]:
rooms_bed_ch = pd.read_csv(f"{output_dir}/census11_rooms_bedrooms_central_heating_oa11_london.csv")
rooms_bed_ch.head()

,date,geography,geography code,Rural Urban,Central Heating: All categories: Type of central heating in household; measures: Value,Central Heating: Does not have central heating; measures: Value,Central Heating: Does have central heating; measures: Value,Central Heating: Occupancy rating (rooms) of -1 or less; measures: Value,Central Heating: Occupancy rating (bedrooms) of -1 or less; measures: Value,Central Heating: Average household size; measures: Value,Central Heating: Average number of rooms per household; measures: Value,Central Heating: Average number of bedrooms per household; measures: Value
0,2011,E00004482,E00004482,Total,121,3,118,8,3,2.4,5.4,2.8
1,2011,E00004497,E00004497,Total,117,1,116,37,23,2.2,3.7,1.7
2,2011,E00004498,E00004498,Total,145,4,141,43,26,2.3,3.9,2.1
3,2011,E00004502,E00004502,Total,120,1,119,32,25,2.1,3.8,1.9
4,2011,E00004505,E00004505,Total,124,0,124,23,8,2.3,4.5,2.4


In [ ]:
rooms_bed_ch.columns

Index(['date', 'geography', 'geography code', 'Rural Urban',
       'Central Heating: All categories: Type of central heating in household; measures: Value',
       'Central Heating: Does not have central heating; measures: Value',
       'Central Heating: Does have central heating; measures: Value',
       'Central Heating: Occupancy rating (rooms) of -1 or less; measures: Value',
       'Central Heating: Occupancy rating (bedrooms) of -1 or less; measures: Value',
       'Central Heating: Average household size; measures: Value',
       'Central Heating: Average number of rooms per household; measures: Value',
       'Central Heating: Average number of bedrooms per household; measures: Value'],
      dtype='object')

In [ ]:
# Segmentation of categories
def room_bed_ch_groups(rbch, fixed_columns):
  d = rbch[fixed_columns].copy()
  total = rbch['All categories: Type of central heating in household']
  d['No central heating'] = rbch['Does not have central heating'] * 100. / total
  d['Central heating'] = rbch['Does have central heating'] * 100. / total
  d['-1 or less (rooms)'] = rbch['Occupancy rating (rooms) of -1 or less']
  d['-1 or less (bedrooms)'] = rbch['Occupancy rating (bedrooms) of -1 or less']
  d['Average household size'] = rbch['Average household size']
  d['Average no of rooms'] = rbch['Average number of rooms per household']
  d['Average no of bedrooms'] = rbch['Average number of bedrooms per household']

  return d#.round(1)

In [ ]:
# Lewisham
for geog, geog_colname in zip(['oa11', 'wd22'], ['OA11CD', 'WD22CD_proposed']):
  IPython.display.display(f"=== {geog} ===")

  rbch = pd.read_csv(f"{output_dir}/lbl_rooms_bedrooms_central_heating_{geog}.csv")
  d = room_bed_ch_groups(rbch, [geog_colname])
  d.to_csv(f"{output_dir}/lbl_rooms_bedrooms_central_heating_groups_{geog}.csv", index=False)
  IPython.display.display(d.head())

'=== oa11 ==='

,OA11CD,No central heating,Central heating,-1 or less (rooms),-1 or less (bedrooms),Average household size,Average no of rooms,Average no of bedrooms
0,E00016403,1.408451,98.591549,38,29,2.6,4.1,2.1
1,E00016437,1.886792,98.113208,63,50,3.4,4.5,2.5
2,E00016439,0.000000,100.000000,81,35,2.1,3.4,1.7
3,E00016442,1.438849,98.561151,50,28,2.5,3.7,2.0
4,E00016399,0.000000,100.000000,46,28,3.3,4.6,2.7


'=== wd22 ==='

,WD22CD_proposed,No central heating,Central heating,-1 or less (rooms),-1 or less (bedrooms),Average household size,Average no of rooms,Average no of bedrooms
0,E05013714,2.434543,97.565457,856,610,83.2,148.0,76.9
1,E05013715,3.281471,96.718529,1379,630,118.9,245.1,125.6
2,E05013716,4.317645,95.682355,2172,990,122.8,223.4,116.6
3,E05013717,2.815297,97.184703,956,551,123.3,248.3,131.5
4,E05013718,3.065624,96.934376,948,577,109.9,225.0,116.9


Car van availability
--

In [ ]:
# Segmentation of categories
def car_van_groups(cv, fixed_columns):
  d = cv[fixed_columns].copy()
  total = cv['All categories: Car or van availability']
  d['No cars or vans'] = cv['No cars or vans in household'] * 100. / total
  d['One car or van'] = cv['1 car or van in household'] * 100. / total
  d['Two cars or vans'] = cv['2 cars or vans in household'] * 100 / total
  d['Three cars or vans'] = cv['3 cars or vans in household'] * 100 / total
  d['Four cars or vans'] = cv['4 or more cars or vans in household'] * 100 / total

  
  return d#.round(1)

In [ ]:
# Lewisham
for geog, geog_colname in zip(['oa11', 'wd22'], ['OA11CD', 'WD22CD_proposed']):
  IPython.display.display(f"=== {geog} ===")

  cv = pd.read_csv(f"{output_dir}/lbl_car_van_availability_{geog}.csv")
  d = car_van_groups(cv, [geog_colname])
  d.to_csv(f"{output_dir}/lbl_car_van_availability_groups_{geog}.csv", index=False)
  IPython.display.display(d.head())

'=== oa11 ==='

,OA11CD,No cars or vans,One car or van,Two cars or vans,Three cars or vans,Four cars or vans
0,E00016403,65.492958,32.394366,1.408451,0.00000,0.704225
1,E00016437,54.716981,40.251572,5.031447,0.00000,0.000000
2,E00016439,76.966292,20.224719,2.808989,0.00000,0.000000
3,E00016442,66.906475,29.496403,3.597122,0.00000,0.000000
4,E00016399,58.156028,34.751773,4.964539,1.41844,0.709220


'=== wd22 ==='

,WD22CD_proposed,No cars or vans,One car or van,Two cars or vans,Three cars or vans,Four cars or vans
0,E05013714,48.828663,40.629306,8.635737,1.538815,0.367478
1,E05013715,45.983027,43.281471,9.278642,1.046676,0.410184
2,E05013716,57.059559,35.957240,5.955852,0.819103,0.208247
3,E05013717,34.125305,45.109845,16.338487,3.515053,0.911310
4,E05013718,41.673319,45.441482,10.601948,1.836181,0.447070


Occupancy rating
--

In [ ]:
# Segmentation of categories
def occupancy_rating_groups(occ, fixed_columns):
  d = occ[fixed_columns].copy()
  total = occ['All categories: Occupancy rating bedrooms']
  d['+2 or more'] = occ['Occupancy rating (bedrooms) of +2 or more'] * 100. / total
  d['+1'] = occ['Occupancy rating (bedrooms) of +1'] * 100. / total
  d['0'] = occ['Occupancy rating (bedrooms) of 0'] * 100 / total
  d['-1'] = occ['Occupancy rating (bedrooms) of -1'] * 100 / total
  d['-2 or less'] = occ['Occupancy rating (bedrooms) of -2 or less'] * 100 / total

  
  return d#.round(1)

In [ ]:
# Lewisham
for geog, geog_colname in zip(['oa11', 'wd22'], ['OA11CD', 'WD22CD_proposed']):
  IPython.display.display(f"=== {geog} ===")

  occ = pd.read_csv(f"{output_dir}/lbl_occupancy_rating_{geog}.csv")
  d = occupancy_rating_groups(occ, [geog_colname])
  d.to_csv(f"{output_dir}/lbl_occupancy_rating_groups_{geog}.csv", index=False)
  IPython.display.display(d.head())

'=== oa11 ==='

,OA11CD,+2 or more,+1,0,-1,-2 or less
0,E00016403,4.929577,32.394366,42.253521,13.380282,7.042254
1,E00016437,6.918239,17.610063,44.025157,24.528302,6.918239
2,E00016439,0.561798,16.853933,62.921348,15.730337,3.932584
3,E00016442,0.000000,35.971223,43.884892,16.546763,3.597122
4,E00016399,5.673759,31.205674,43.262411,17.021277,2.836879


'=== wd22 ==='

,WD22CD_proposed,+2 or more,+1,0,-1,-2 or less
0,E05013714,12.999541,28.847037,44.143316,11.667432,2.342673
1,E05013715,20.113154,30.367751,40.608204,7.609618,1.301273
2,E05013716,12.550326,25.197834,48.507566,11.467444,2.276829
3,E05013717,30.756713,27.925142,32.351505,7.306753,1.659886
4,E05013718,24.445154,32.572250,33.769759,8.015328,1.197509


Reference geographies accommodation type
--

In [ ]:
# Reference geographies
acc_type_ref = pd.read_csv(f"{output_dir}/references_accommodation_type.csv")
d = accomm_type_groups(acc_type_ref, ['Name', 'Code'])
d.to_csv(f"{output_dir}/references_accommodation_type_groups.csv", index=False)
IPython.display.display(d.head())



,Name,Code,Detached houses,Semi-detached houses,Terraced,Flats,Caravan or mobile
0,England,E92000001,22.255383,30.708059,24.487699,22.113919,0.434940
1,London,E12000007,6.236082,18.587529,22.933644,52.159551,0.083194
2,Lewisham,E09000023,3.260915,13.141455,28.159539,55.394252,0.043839


Reference geographies Tenure Households
--

In [ ]:
# Reference geographies
tenure_hh_ref = pd.read_csv(f"{output_dir}/references_tenure_households.csv")
d = tenure_groups(tenure_hh_ref, ['Name', 'Code'])
d.to_csv(f"{output_dir}/references_tenure_households_groups.csv", index=False)
IPython.display.display(d.head())

,Name,Code,Owned,Shared ownership,Social rented,Private rented,Rent free
0,England,E92000001,63.340393,0.787550,17.692448,16.842052,1.337556
1,London,E12000007,48.258528,1.289215,24.064647,25.077820,1.309790
2,Lewisham,E09000023,42.404665,1.236961,31.054948,24.305071,0.998355


Reference geographies Household composition
--

In [ ]:
# Reference geographies
hhcomp_ref = pd.read_csv(f"{output_dir}/references_household_compositions.csv")
d = household_comp_groups(hhcomp_ref, ['Name', 'Code'])
d.to_csv(f"{output_dir}/references_household_compositions_groups.csv", index=False)
IPython.display.display(d.head())

,Name,Code,One person household,One family household,Other household types
0,England,E92000001,30.215210,61.781964,8.002826
1,London,E12000007,31.552462,53.482807,14.964731
2,Lewisham,E09000023,34.180083,51.330422,14.489495


Reference geographies Rooms bedrooms central heating
--

In [ ]:
# Reference geographies
rbch_ref = pd.read_csv(f"{output_dir}/references_rooms_bedrooms_central_heating.csv")
d = room_bed_ch_groups(rbch_ref, ['Name', 'Code'])
d.to_csv(f"{output_dir}/references_rooms_bedrooms_central_heating_groups.csv", index=False)
IPython.display.display(d.head())

,Name,Code,No central heating,Central heating,-1 or less (rooms),-1 or less (bedrooms),Average household size,Average no of rooms,Average no of bedrooms
0,England,E92000001,2.694788,97.305212,1928596,1024473,2.4,5.4,2.7
1,London,E12000007,2.829336,97.170664,707437,370531,2.5,4.7,2.5
2,Lewisham,E09000023,3.256066,96.743934,25722,14018,2.4,4.5,2.3


Reference geographies Car van availability
--

In [ ]:
# Reference geographies
cv_ref = pd.read_csv(f"{output_dir}/references_car_van_availability.csv")
d = car_van_groups(cv_ref, ['Name', 'Code'])
d.to_csv(f"{output_dir}/references_car_van_availability_groups.csv", index=False)
IPython.display.display(d.head())

,Name,Code,No cars or vans,One car or van,Two cars or vans,Three cars or vans,Four cars or vans
0,England,E92000001,25.795024,42.159366,24.663474,5.456397,1.925740
1,London,E12000007,41.554780,40.537718,14.042704,2.927555,0.937244
2,Lewisham,E09000023,48.145851,40.477729,9.328027,1.615112,0.433281


Reference geographies Occupancy rating
--

In [ ]:
# Reference geographies
occ_rating_ref = pd.read_csv(f"{output_dir}/references_occupancy_rating.csv")
d = occupancy_rating_groups(occ_rating_ref, ['Name', 'Code'])
d.to_csv(f"{output_dir}/references_occupancy_rating_groups.csv", index=False)
IPython.display.display(d.head())

,Name,Code,+2 or more,+1,0,-1,-2 or less
0,England,E92000001,34.259570,34.419627,26.677482,3.945635,0.697686
1,London,E12000007,21.111497,28.266139,39.277864,9.225629,2.118871
2,Lewisham,E09000023,18.072891,27.967715,41.884384,10.081746,1.993264
